In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch import nn, optim
from torch.utils.data import DataLoader
from utils.dataloader import MiniImagenet
from common.cnn_models import FewshotClassifier
from maml.models import MAML
from maml.utils import fast_adapt
from tqdm import tqdm

In [2]:
torch.manual_seed(777)
torch.cuda.manual_seed_all(777)
np.random.seed(777)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
N = 5
K = 1
Q = 15
batch_size = 8
meta_lr=0.003
fast_lr=0.5
adaptation_steps=1

In [4]:
root_path = './datasets/miniimagenet/pkl_file/' 
train_dataset = MiniImagenet(path=root_path, N=N, K=K, Q=Q, mode='train')
train_loader = DataLoader(train_dataset, batch_size=batch_size,\
                          shuffle=True, num_workers=1)
val_dataset = MiniImagenet(path=root_path, N=N, K=K, Q=Q, mode='validation', total_iter=1000)
val_loader = DataLoader(val_dataset, batch_size=batch_size,\
                        shuffle=True, num_workers=1)
test_dataset = MiniImagenet(path=root_path, N=N, K=K, Q=Q, mode='test', total_iter=5000)
test_loader = DataLoader(test_dataset, batch_size=batch_size,\
                         shuffle=True, num_workers=1)

In [5]:
# Create model
model = FewshotClassifier(output_size=N)
model.to(device)
maml = MAML(model, lr=fast_lr, first_order=False)
maml.to(device)
opt = optim.Adam(maml.parameters(), meta_lr)
loss = nn.CrossEntropyLoss(reduction='mean')

In [6]:
iteration = 0
for task_batch in train_loader:
    torch.cuda.empty_cache()
    opt.zero_grad()
    meta_train_error = []
    meta_train_accuracy = []
    for batch in zip(*task_batch):
        learner = maml.clone()
        evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                           learner,
                                                           loss,
                                                           adaptation_steps,
                                                           device)
        evaluation_error.backward()
        meta_train_error.append(evaluation_error.item())
        meta_train_accuracy.append(evaluation_accuracy.item())
        iteration += 1 
    # Average the accumulated gradients and optimize
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / len(task_batch))
    opt.step()

    if iteration % 1000 == 0:
        print('Iteration', iteration)
        print('Meta Train Error : ', round(np.mean(meta_train_error),4),\
              " std : ", round(np.std(meta_train_error), 4))
        print('Meta Train Accuracy : ', round(np.mean(meta_train_accuracy), 4),\
              " std : ", round(np.std(meta_train_accuracy), 4))
    
    if iteration % 5000 == 0:
        # Compute meta-validation loss
        meta_valid_error = []
        meta_valid_accuracy = []
        for task_batch in val_loader:
            for batch in zip(*task_batch):
                learner = maml.clone()
                evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                                   learner,
                                                                   loss,
                                                                   adaptation_steps,
                                                                   device)
                meta_valid_error.append(evaluation_error.item())
                meta_valid_accuracy.append(evaluation_accuracy.item())
        print('Valid Result')
        print('Meta Valid Error : ', round(np.mean(meta_valid_error),4),\
              " std : ", round(np.std(meta_valid_error), 4))
        print('Meta Valid Accuracy : ', round(np.mean(meta_valid_accuracy), 4),\
              " std : ", round(np.std(meta_valid_accuracy), 4))

  0%|          | 0/7500 [00:00<?, ?it/s]/root/anaconda3/envs/graph_rec/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448234945/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
  2%|▏         | 125/7500 [00:32<31:02,  3.96it/s]

Iteration 1000
Meta Train Error 3.548592448234558
Meta Train Error 0.4633333422243595


  3%|▎         | 250/7500 [01:03<31:33,  3.83it/s]

Iteration 2000
Meta Train Error 3.1803328096866608
Meta Train Error 0.5100000128149986


  5%|▌         | 375/7500 [01:35<29:59,  3.96it/s]

...

Iteration 59000
Meta Train Error 2.196537896990776
Meta Train Error 1.1033333465456963


100%|█████████▉| 7499/7500 [35:20<00:00,  4.04it/s]

Iteration 60000
Meta Train Error 2.162126451730728
Meta Train Error 1.1533333659172058


100%|██████████| 7500/7500 [35:41<00:00,  3.50it/s]

Valid Result
Meta Valid Error :  1.3371518250107766
Meta Valid Accuracy :  0.4474800105243921  std :  0.18822809740505483


In [9]:
meta_train_accuracy

4.613333463668823

In [10]:
meta_test_error = []
meta_test_accuracy = []
for task_batch in test_loader:
    for batch in zip(*task_batch):
        learner = maml.clone()
        evaluation_error, evaluation_accuracy = fast_adapt(batch,
                                                           learner,
                                                           loss,
                                                           adaptation_steps,
                                                           device)
        meta_test_error.append(evaluation_error.item())
        meta_test_accuracy.append(evaluation_accuracy.item())
        torch.cuda.empty_cache()
print('Valid Result')
print('Meta Valid Error : ', round(np.mean(meta_test_error),4),\
        " std : ", round(np.std(meta_test_error), 4))
print('Meta Valid Accuracy : ', round(np.mean(meta_test_accuracy), 4),\
        " std : ", round(np.std(meta_test_accuracy), 4))

Valid Result
Meta Valid Error :  1.3157  std :  0.1907
Meta Valid Accuracy :  0.4559  std :  0.0972
